# Movielenes 데이터를 활용한 영화 추천 시스템
***

Movielens 데이터를 활용해 영화 추천 시스템을 만들어보자.
해당 모델을 다음 과정을 거쳐 만들어보자.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. 
  MovieLens 1M Dataset 사용을 권장합니다.

* 별점 데이터는 대표적인 explicit 데이터입니다. 
  하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.

* 별점을 시청횟수로 해석해서 생각하겠습니다.

* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
 
MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

```
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip
```

## 1. 데이터 준비와 전처리
***

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있다.

In [1]:
import pandas as pd
import os
from pandas import Series

rating_file_path=os.getenv('HOME') + '/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 평점 3점 이상만 남긴다.

ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [4]:
# 영화 제목을 보기 위해 메타 데이터 불러오기.
movie_file_path=os.getenv('HOME') + '/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [6]:
movies['genre'] = movies['genre'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy


## 2. 데이터 분석
***
다음 과 같은 기준으로 데이터를 분석해보자

* ratings에 있는 유니크한 영화 개수
* rating에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [7]:
# 두 데이터 병합

df = pd.merge(ratings, movies, how = 'inner', on = 'movie_id')
df

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,one little indian (1973),comedy|drama|western
836474,5854,3026,4,958346883,slaughterhouse (1987),horror
836475,5854,690,3,957744257,"promise, the (versprechen, das) (1994)",romance
836476,5938,2909,4,957273353,"five wives, three secretaries and me (1998)",documentary


In [8]:
# 불필요한 컬럼 제거
df_cols = ['user_id', 'movie_id', 'count', 'title']

df = df[df_cols]
df

,user_id,movie_id,count,title
0,1,1193,5,one flew over the cuckoo's nest (1975)
1,2,1193,5,one flew over the cuckoo's nest (1975)
2,12,1193,4,one flew over the cuckoo's nest (1975)
3,15,1193,4,one flew over the cuckoo's nest (1975)
4,17,1193,5,one flew over the cuckoo's nest (1975)
...,...,...,...,...
836473,5851,3607,5,one little indian (1973)
836474,5854,3026,4,slaughterhouse (1987)
836475,5854,690,3,"promise, the (versprechen, das) (1994)"
836476,5938,2909,4,"five wives, three secretaries and me (1998)"


In [9]:
# df 에 있는 유니크한 영화 개수
df['movie_id'].nunique()

3628

In [10]:
# df 에 있는 유니크한 사용자 수

df['user_id'].nunique()

6039

In [11]:
# 가장 인기 있는 영화 30개 (인기순)

df_count = df.groupby('user_id')['movie_id'].count()
df_count.sort_values(ascending=False).head(30)

user_id
4169    1968
4277    1715
1680    1515
3618    1146
1015    1145
5831    1136
2909    1119
1941    1116
424     1106
1980    1054
3224    1034
1285    1034
3539    1029
3391    1012
3032    1003
1181     981
3841     976
4344     973
4448     970
1088     966
549      956
889      943
4725     936
678      933
1448     930
2063     906
3292     897
1698     883
4808     868
2116     855
Name: movie_id, dtype: int64

## 3. 내가 선호하는 영화를 5가지 고르기
***

내가 선호하는 영화를 5가지 골라서 rating 에 추가해 주기

In [17]:
my_favorite = ['toy story (1995)', 'jumanji (1995)', 'grumpier old men (1995)', 'waiting to exhale (1995', 'father of the bride part ii (1995)']

my_favorite_id = ['1', '2', '3', '4', '5']    

my_favorite_list = pd.DataFrame({'user_id' : ['7777.0']*5, 'movie_id' : my_favorite_id, 'count' : [5]*5, 'title' : my_favorite})

if not df.isin({'user_id' : [7777.0]})['user_id'].any() : 
    
    df = df.append(my_favorite_list)
    
df.tail(5)

,user_id,movie_id,count,title
0,7777.0,1,5,toy story (1995)
1,7777.0,2,5,jumanji (1995)
2,7777.0,3,5,grumpier old men (1995)
3,7777.0,4,5,waiting to exhale (1995
4,7777.0,5,5,father of the bride part ii (1995)


## 4. CSR Matrix 만들기
***

In [24]:
# 모델활용을 위한 전처리

user_unique = df['user_id'].unique()
title_unique = df['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}

title_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [25]:
# 인덱싱이 잘 되었는지 확인

print(user_to_idx['7777.0'])
print(title_to_idx['toy story (1995)'])

6039
40


In [28]:
# indexing 을 통해 데이터 컬럼 내 값을 바꾸는 코드

temp_user_data = df['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(df) :
    print('user_id column indexing OK!!')
    
    df['user_id'] = temp_user_data
    
else :
    print('user_id_column indexing Fail!!')
    
temp_title_data = df['title'].map(title_to_idx.get).dropna()

if len(temp_title_data) == len(df) : 
    
    print('title column indexing OK!!')
    
    df['title'] = temp_title_data
    
else :
    print('title column indexing Fail!!')
    
    
df

user_id_column indexing Fail!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,1,1193,5,0
2,2,1193,4,0
3,3,1193,4,0
4,4,1193,5,0
...,...,...,...,...
0,6039,1,5,40
1,6039,2,5,513
2,6039,3,5,1862
3,6039,4,5,3628


In [36]:
# CSR Matrix 모델 생성

from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_title = df['title'].nunique()

In [38]:
csr_df = csr_matrix((df['count'], (df.user_id, df.title)), shape=(num_user, num_title))
csr_df

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델링 및 훈련
***

In [39]:
from implicit.als import AlternatingLeastSquares
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [40]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [41]:
csr_df_transpose = csr_df.T
csr_df_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [42]:
als_model.fit(csr_df_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [47]:
종함, 영화제목 = user_to_idx['7777.0'], title_to_idx['toy story (1995)']

종함_vector, 영화제목_vector = als_model.user_factors[종함], als_model.item_factors[영화제목]

In [48]:
종함_vector

array([ 0.18722868, -0.6117833 ,  0.34382686,  0.75006366, -0.4766836 ,
       -0.4932248 ,  0.390701  , -0.29227155,  0.16208601, -0.23412865,
       -0.27489257,  0.2117479 ,  0.5687585 ,  0.30369237, -0.20647605,
       -0.58376145, -0.17820446,  0.7901568 , -0.15515614, -0.18393016,
       -0.3188245 , -0.03854455,  0.05642527,  0.07141355, -0.05272022,
       -0.39846653, -0.21723583, -0.26510632, -0.18242274,  0.33319163,
        0.53998345,  0.48012573,  0.37454942, -0.06461246,  0.63723093,
       -0.7283513 ,  0.00287801,  0.43111894,  0.14366071,  0.01378844,
       -0.8357798 ,  0.37970355, -0.07679348, -0.2284115 , -0.77961284,
        0.35686162,  0.03982905,  0.2682473 , -0.08034252,  0.35125828,
        0.07543333,  0.17004041,  0.73729867, -0.43061435, -0.8679469 ,
       -0.14253758,  0.14155845, -0.24021716, -0.22310658,  0.38326922,
       -0.06382319, -0.4999149 ,  0.77588445, -0.95324624, -0.34567988,
        0.20735946, -0.20443335,  0.25121218, -0.10122638,  0.18

In [49]:
영화제목_vector

array([-0.00032247, -0.0258317 , -0.00840501,  0.05457194, -0.01286818,
       -0.00936423,  0.02962062,  0.00499933,  0.00909372, -0.007734  ,
       -0.00208191,  0.00162446,  0.0107869 , -0.00566979,  0.00627036,
       -0.03530474,  0.02035915,  0.03788102,  0.0111387 ,  0.00804888,
       -0.00196372, -0.00216968, -0.01380303, -0.00165745,  0.00040435,
       -0.00216799,  0.0028774 , -0.04242567, -0.00580891,  0.01334452,
        0.02509223, -0.00418619, -0.01276184, -0.00703111,  0.01481431,
        0.0038033 ,  0.00829662,  0.03234744,  0.03022364,  0.02324366,
       -0.01516123,  0.02254579,  0.01152043, -0.00750755, -0.02685377,
        0.03744654,  0.00884402,  0.01910251,  0.01412138,  0.0379901 ,
       -0.00697373, -0.00215771,  0.01298034, -0.01332936, -0.01158918,
        0.0131913 ,  0.01038891,  0.01505379, -0.00186662,  0.02218537,
       -0.00950174, -0.0318371 , -0.00102545, -0.04231222,  0.00808172,
       -0.01199393, -0.00063074,  0.00311384, -0.00681894,  0.02

In [50]:
# 종함 과 영화제목를 내적하는 코드

np.dot(종함_vector, 영화제목_vector)

0.47061086

In [55]:
jumanji = title_to_idx['jumanji (1995)']
jumanji_vactor = als_model.item_factors[jumanji]
np.dot(종함_vector, jumanji_vactor)

0.27456754

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악
***

In [62]:
favorite_movie = 'grumpier old men (1995)'
title_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(title_id, N = 15)
similar_movie

[(1862, 0.9999999),
 (1045, 0.7158471),
 (1207, 0.53996116),
 (624, 0.51889855),
 (1181, 0.51563096),
 (560, 0.49399033),
 (1172, 0.49290985),
 (635, 0.49166277),
 (701, 0.4868525),
 (1170, 0.48671478),
 (1511, 0.48221675),
 (1211, 0.47817886),
 (1225, 0.4712305),
 (523, 0.4658309),
 (1237, 0.46265775)]

In [64]:
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['grumpier old men (1995)',
 'grumpy old men (1993)',
 'milk money (1994)',
 'naked gun 33 1/3: the final insult (1994)',
 "she's the one (1996)",
 'nine months (1995)',
 'forces of nature (1999)',
 'robin hood: men in tights (1993)',
 'santa claus: the movie (1985)',
 'multiplicity (1996)',
 'mirror has two faces, the (1996)',
 'fools rush in (1997)',
 "butcher's wife, the (1991)",
 'michael (1996)',
 'speechless (1994)']

In [65]:
# 위 코드를 반복을 위해 함수로 생성

def get_similar_movie(movie_title : str) : 
    title_id = title_to_idx[movie_title]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    
    return similar_title

In [67]:
# 확인

get_similar_movie('father of the bride part ii (1995)')

['father of the bride part ii (1995)',
 'nine months (1995)',
 'flintstones, the (1994)',
 'home alone 2: lost in new york (1992)',
 'mighty ducks, the (1992)',
 'multiplicity (1996)',
 'first wives club, the (1996)',
 'edtv (1999)',
 'jack (1996)',
 'sister act 2: back in the habit (1993)']

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천하기
***

In [68]:
user = user_to_idx['7777.0']

movie_recommended = als_model.recommend(user, csr_df, N=20, filter_already_liked_items = True)
movie_recommended

[(50, 0.48998684),
 (322, 0.31304342),
 (545, 0.2760967),
 (458, 0.24219733),
 (4, 0.23448232),
 (1045, 0.21367711),
 (596, 0.20057997),
 (33, 0.19708106),
 (812, 0.18228376),
 (507, 0.17857972),
 (60, 0.17356683),
 (173, 0.17260715),
 (330, 0.16766842),
 (119, 0.16012168),
 (678, 0.15550396),
 (1130, 0.15524589),
 (32, 0.15506662),
 (1982, 0.14853685),
 (624, 0.14692411),
 (1514, 0.13916172)]

## 8) 내가 가장 좋아할 만한 영화들을 추천받기
***

In [69]:
[idx_to_title[i[0]] for i in movie_recommended]

['toy story 2 (1999)',
 'babe (1995)',
 'santa clause, the (1994)',
 'mask, the (1994)',
 "bug's life, a (1998)",
 'grumpy old men (1993)',
 'hook (1991)',
 'aladdin (1992)',
 'nutty professor, the (1996)',
 'home alone (1990)',
 'star wars: episode i - the phantom menace (1999)',
 'dragonheart (1996)',
 'lion king, the (1994)',
 'twister (1996)',
 'willy wonka and the chocolate factory (1971)',
 'indian in the cupboard, the (1995)',
 'hercules (1997)',
 'space jam (1996)',
 'naked gun 33 1/3: the final insult (1994)',
 'six days seven nights (1998)']

### 회고
***

1. 앞선 프로젝트들을 실행할 때 노드에 있는 코드를 완전히 이해한다기보다 필사느낌이 더 강했는데, 이번 노드를 진행하면서 각 코드들에 대해 완전히 이해를 못하고 넘어가는게 얼마나 큰 문제가 되는지 알았다.

2. 위 문제를 체감하고 다시 각 셀들에 대한 구조와 쓰이는 서식을 하나씩 이해하려하니, 너무 어지럽다. 역시 한번에 다 떄려넣으려하지말고 꾸준히, 꾸준히 공부해야겠다는 것을 느꼈다.

3. 전처리 과정에서 데이터 병합 후 전처리하는 과정에서 title 컬럼의 value 값들을 슬라이싱해서 title 뒤에 년도를 지우는 것으로 바꿔주는것도 나쁘지 않지 않을까 싶다. 